In [1]:
import tifffile as tf

import json, os, csv
import pandas as pd
import numpy as np
from tqdm import tqdm


from matplotlib import pyplot as plt
from shapely.geometry import Polygon
from PIL import Image, ImageDraw
import random


In [2]:
# Load static paths
root = '/media/sam/New Volume/Xenium_Data'
baysor_root = '/media/sam/Data2/baysor_rbpms_consolidated'
xen_roots = ['output-XETG00230__0018429__Region_1__20240105__233208',
             'output-XETG00230__0018432__Region_2__20240105__233208',
             'BudoffRun2_Slide 3_4/BudoffRun2_Slide 3_4/output-XETG00230__0018336__Region_1__20240124__002923',
             'BudoffRun2_Slide 3_4/BudoffRun2_Slide 3_4/output-XETG00230__0018521__Region_1__20240124__002923',
             'BudoffRun3_Slide 5_6/BudoffRun3_Slide 5_6/output-XETG00230__0018624__Region_1__20240127__000149',
             'BudoffRun3_Slide 5_6/BudoffRun3_Slide 5_6/output-XETG00230__0022826__Region_1__20240127__000149',
             'BudoffRun4_Slide 7_8/BudoffRun4_Slide 7_8/output-XETG00230__0018300__Region_1__20240206__235339',
             'BudoffRun4_Slide 7_8/BudoffRun4_Slide 7_8/output-XETG00230__0022825__Region_1__20240206__235339']

pix_size = 0.2125

# Extract slide_ids
slides = [xen_root[-33:-28] for xen_root in xen_roots]

files_dict = {}

for (xen_root, slide) in zip(xen_roots, slides):
    files_dict[slide] = {}
    files_dict[slide]['xen_root'] = os.path.join(root, xen_root)
    files_dict[slide]['dapi_path'] = os.path.join(files_dict[slide]['xen_root'], 'morphology_focus.ome.tif')
    files_dict[slide]['slice_root'] = os.path.join(baysor_root, slide)
    files_dict[slide]['slices'] = os.listdir(files_dict[slide]['slice_root'])
    files_dict[slide]['slice_paths'] = [os.path.join(files_dict[slide]['slice_root'], s)for s in files_dict[slide]['slices']]
    files_dict[slide]['seg_paths'] = [os.path.join(s, "segmentation_polygons_2d.json")for s in files_dict[slide]['slice_paths']]
    
files_dict

{'18429': {'xen_root': '/media/sam/New Volume/Xenium_Data/output-XETG00230__0018429__Region_1__20240105__233208',
  'dapi_path': '/media/sam/New Volume/Xenium_Data/output-XETG00230__0018429__Region_1__20240105__233208/morphology_focus.ome.tif',
  'slice_root': '/media/sam/Data2/baysor_rbpms_consolidated/18429',
  'slices': ['18429_R1.01_region0',
   '18429_R1.02_region0',
   '18429_R1.03_region0',
   '18429_R1.04_region0',
   '18429_R1.05_region0',
   '18429_R1.06_region0',
   '18429_R1.07_region0',
   '18429_R1.08_region0'],
  'slice_paths': ['/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.01_region0',
   '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.02_region0',
   '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.03_region0',
   '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.04_region0',
   '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.05_region0',
   '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.06_region0',
   

In [3]:
def find_frame(baysor_seg, pix_size):
    x_max = -np.inf
    y_max = -np.inf
    x_min = np.inf
    y_min = np.inf
    
    # First pass to find the exact bounds
    for cell_feature in tqdm(baysor_seg['features']):
        cell_nodes = cell_feature['geometry']['coordinates'][0]
        # Extract and transform coordinates
        coords = np.array(cell_nodes) / pix_size
        
        min_x = np.min(coords[:, 0])
        max_x = np.max(coords[:, 0])
        min_y = np.min(coords[:, 1])
        max_y = np.max(coords[:, 1])
        
        x_min = min(x_min, min_x)
        x_max = max(x_max, max_x)
        y_min = min(y_min, min_y)
        y_max = max(y_max, max_y)
    
    # Convert to integers carefully
    x_min = int(np.floor(x_min))
    x_max = int(np.ceil(x_max))
    y_min = int(np.floor(y_min))
    y_max = int(np.ceil(y_max))
    
    # Shape is in (height, width) format
    shape = (y_max - y_min + 1, x_max - x_min + 1)
    bounds = (x_min, y_min, x_max, y_max)
    
    return shape, bounds

def create_cell_mask(baysor_seg, dapi, pix_size):
    # Get frame dimensions and cropping coordinates
    shape, (x_min, y_min, x_max, y_max) = find_frame(baysor_seg, pix_size)
    
    print(f"Original DAPI shape: {dapi.shape}")
    print(f"Calculated shape: {shape}")
    print(f"Bounds: x({x_min}, {x_max}), y({y_min}, {y_max})")
    
    # Ensure bounds are within DAPI image
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(dapi.shape[1] - 1, x_max)
    y_max = min(dapi.shape[0] - 1, y_max)
    
    # Recalculate shape based on actual cropping
    shape = (y_max - y_min + 1, x_max - x_min + 1)
    
    # Crop DAPI image to the region of interest
    dapi_cropped = dapi[y_min:y_max+1, x_min:x_max+1]
    print(f"Cropped DAPI shape: {dapi_cropped.shape}")
    
    # Create a two-channel image array (channel 0: DAPI, channel 1: cell outlines)
    cell_mask = np.zeros((shape[0], shape[1], 2), dtype=np.uint16)
    print(f"Cell mask shape: {cell_mask.shape}")
    
    # Add the cropped DAPI to channel 0
    cell_mask[:, :, 0] = dapi_cropped
    
    # Create a temporary PIL image for drawing cell outlines
    outline_mask = Image.new('L', (shape[1], shape[0]), 0)  # Note width, height order for PIL
    draw = ImageDraw.Draw(outline_mask)
    
    # Initialize DataFrame for DAPI statistics
    dapi_df = pd.DataFrame(columns=['cell', 'dapi_max', 'dapi_min', 'dapi_mean', 'dapi_sd'])
    
    # Iterate over each cell and draw its outline
    for cell_feature in tqdm(baysor_seg['features']):
        cell_nodes = cell_feature['geometry']['coordinates'][0]
        
        # Extract and transform coordinates
        polygon_coords = [
            ((node[0] / pix_size - x_min), (node[1] / pix_size - y_min))
            for node in cell_nodes
        ]
        
        # Calculate local bounding box for DAPI statistics
        min_x_local = max(0, int(np.floor(min([coord[0] for coord in polygon_coords]))))
        max_x_local = min(shape[1]-1, int(np.ceil(max([coord[0] for coord in polygon_coords]))))
        min_y_local = max(0, int(np.floor(min([coord[1] for coord in polygon_coords]))))
        max_y_local = min(shape[0]-1, int(np.ceil(max([coord[1] for coord in polygon_coords]))))
        
        # Create mask for DAPI measurements
        polygon_mask = Image.new('L', (max_x_local-min_x_local+1, max_y_local-min_y_local+1), 0)
        shifted_coords = [(x-min_x_local, y-min_y_local) for x, y in polygon_coords]
        ImageDraw.Draw(polygon_mask).polygon(shifted_coords, outline=1, fill=1)
        polygon_mask = np.array(polygon_mask)
        
        # Extract DAPI statistics
        dapi_region = dapi_cropped[min_y_local:max_y_local+1, min_x_local:max_x_local+1]
        masked_dapi = dapi_region * polygon_mask
        
        # Only calculate statistics if we have valid pixels
        valid_pixels = polygon_mask > 0
        if np.any(valid_pixels):
            dapi_df.loc[len(dapi_df)] = [
                cell_feature['id'],
                np.max(masked_dapi),
                np.min(masked_dapi[valid_pixels]),
                np.mean(masked_dapi[valid_pixels]),
                np.std(masked_dapi[valid_pixels])
            ]
        
        # Draw the cell outline
        draw.line(polygon_coords + [polygon_coords[0]], fill=65535, width=1)
    
    # Add the outline mask to channel 1
    cell_mask[:, :, 1] = np.array(outline_mask)
    
    return cell_mask, dapi_df

def save_cell_mask(cell_mask, output_path):
    """
    Save the cell mask as a multi-channel TIFF file
    """
    # Ensure the array is in the correct data type
    cell_mask = cell_mask.astype(np.uint16)
    
    # Save as TIFF with channels along the first axis (as expected by tifffile)
    cell_mask_transposed = np.transpose(cell_mask, (2, 0, 1))
    tf.imwrite(output_path, cell_mask_transposed)


In [4]:
for slide in slides:
    print(f'loading dapi slide {slide}')
    dapi = tf.imread(files_dict[slide]['dapi_path'], series=0, level=0)
    print(f'loading segmentations from {slide}')
    for output_path in files_dict[slide]['slice_paths']:
        seg_path = os.path.join(output_path, "segmentation_polygons_2d.json")
        csv_path = os.path.join(output_path,"dapi_statistics.csv")
        
        if not os.path.exists(csv_path):
            if os.path.exists(seg_path):
                with open(seg_path, 'r') as file:
                    baysor_seg = json.load(file)
                print(f'loaded {seg_path}')

                # Generate the mask and statistics
                cell_mask, dapi_df = create_cell_mask(baysor_seg, dapi, pix_size)

                print(f'image and statistics computed for {seg_path}')

                # Save the result
                save_cell_mask(cell_mask, os.path.join(output_path, "dapi_baysor_segs.tif"))

                # Save the DAPI statistics if needed
                dapi_df.to_csv(csv_path)
                print(f'image and statistics saved for {seg_path}')
            else:
                print(f'{seg_path} not found')
        else:
            print(f'{csv_path} already exists')

loading dapi slide 18429
loading segmentations from 18429
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.01_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.02_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.03_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.04_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.05_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.06_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.07_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.08_region0/dapi_statistics.csv already exists
loading dapi slide 18432
loading segmentations from 18432
/media/sam/Data2/baysor_rbpms_consolidated/18432/184

100%|████████████████████████████████| 221652/221652 [00:02<00:00, 86453.34it/s]


Original DAPI shape: (112193, 54055)
Calculated shape: (21643, 21994)
Bounds: x(25545, 47538), y(89445, 111087)
Cropped DAPI shape: (21643, 21994)
Cell mask shape: (21643, 21994, 2)


100%|██████████████████████████████████| 221652/221652 [08:44<00:00, 422.74it/s]


image and statistics computed for /media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.09_region0/segmentation_polygons_2d.json
image and statistics saved for /media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.09_region0/segmentation_polygons_2d.json
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.11_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.12_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.13_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.14_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.15_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.16_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.17_region0/dapi_statistics.csv already exists
/media/sa

100%|████████████████████████████████| 177927/177927 [00:01<00:00, 95951.87it/s]


Original DAPI shape: (108768, 51189)
Calculated shape: (23875, 19748)
Bounds: x(25618, 45365), y(31345, 55219)
Cropped DAPI shape: (23875, 19748)
Cell mask shape: (23875, 19748, 2)


100%|██████████████████████████████████| 177927/177927 [05:28<00:00, 541.45it/s]


image and statistics computed for /media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.09_region0/segmentation_polygons_2d.json
image and statistics saved for /media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.09_region0/segmentation_polygons_2d.json
/media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.10_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.11_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.12_region0/dapi_statistics.csv already exists
loading dapi slide 18300
loading segmentations from 18300
/media/sam/Data2/baysor_rbpms_consolidated/18300/18300_R4.13_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18300/18300_R4.14_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18300/18300_R5.01_region0/dapi_statistics.csv already exists
/media/sam/Data2/baysor_rbpms_consolidated/18300/18300_